# T1124 - System Time Discovery
An adversary may gather the system time and/or time zone from a local or remote system. The system time is set and stored by the Windows Time Service within a domain to maintain time synchronization between systems and services in an enterprise network. (Citation: MSDN System Time) (Citation: Technet Windows Time Service)

System time information may be gathered in a number of ways, such as with [Net](https://attack.mitre.org/software/S0039) on Windows by performing <code>net time \\hostname</code> to gather the system time on a remote system. The victim's time zone may also be inferred from the current system time or gathered by using <code>w32tm /tz</code>. (Citation: Technet Windows Time Service)

This information could be useful for performing other techniques, such as executing a file with a [Scheduled Task/Job](https://attack.mitre.org/techniques/T1053) (Citation: RSA EU12 They're Inside), or to discover locality information based on time zone to assist in victim targeting (i.e. [System Location Discovery](https://attack.mitre.org/techniques/T1614)). Adversaries may also use knowledge of system time as part of a time bomb, or delaying execution until a specified date/time.(Citation: AnyRun TimeBomb)

## Atomic Tests

### Atomic Test #1 - System Time DiscoveryIdentify the system time. Upon execution, the local computer system time and timezone will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
net time \\localhost
w32tm /tz
```

In [ ]:
Invoke-AtomicTest T1124 -TestNumbers 1

### Atomic Test #2 - System Time Discovery - PowerShellIdentify the system time via PowerShell. Upon execution, the system time will be displayed.
**Supported Platforms:** windows#### Attack Commands: Run with `powershell`
```powershell
Get-Date
```

In [ ]:
Invoke-AtomicTest T1124 -TestNumbers 2

### Atomic Test #3 - System Time Discovery in macOSIdentify system time. Upon execution, the local computer system time and timezone will be displayed. 
**Supported Platforms:** macos#### Attack Commands: Run with `sh`
```sh
date
```

In [ ]:
Invoke-AtomicTest T1124 -TestNumbers 3

### Atomic Test #4 - System Time Discovery W32tm as a Delayidentifies DCRat delay time tactics using w32tm.
https://research.splunk.com/endpoint/b2cc69e7-11ba-42dc-a269-59c069a48870/
https://blogs.blackberry.com/en/2022/05/dirty-deeds-done-dirt-cheap-russian-rat-offers-backdoor-bargains
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
W32tm /stripchart /computer:localhost /period:5 /dataonly /samples:2
```

In [ ]:
Invoke-AtomicTest T1124 -TestNumbers 4

### Atomic Test #5 - System Time with Windows time CommandDisplays the current system time via the Windows builtin time command: https://learn.microsoft.com/en-us/windows-server/administration/windows-commands/time
Recently observed in use in the wild during an incident involving Ursnif malware:
https://github.com/The-DFIR-Report/Sigma-Rules/blob/dc72f0b557fc63347379be0a33439788256761c8/rules/windows/process_creation/proc_creation_win_system_time_lookup.yml
https://thedfirreport.com/2023/01/09/unwrapping-ursnifs-gifts/
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
time
```

In [ ]:
Invoke-AtomicTest T1124 -TestNumbers 5

## Detection
Command-line interface monitoring may be useful to detect instances of net.exe or other command-line utilities being used to gather system time or time zone. Methods of detecting API use for gathering this information are likely less useful due to how often they may be used by legitimate software.


## Shield Active Defense
### Software Manipulation 
 Make changes to a system's software properties and functions to achieve a desired effect. 

 Software Manipulation allows a defender to alter or replace elements of the operating system, file system, or any other software installed and executed on a system.
#### Opportunity
There is an opportunity for the defender to observe the adversary and control what they can see, what effects they can have, and/or what data they can access.
#### Use Case
If the defender knows the specific regions an adversary is targeting, they can alter the output of commands which return systems times to return data consistent with what an adversary would want to see.
#### Procedures
Hook the Win32 Sleep() function so that it always performs a Sleep(1) instead of the intended duration. This can increase the speed at which dynamic analysis can be performed when a normal malicious file sleeps for long periods before attempting additional capabilities.
Hook the Win32 NetUserChangePassword() and modify it such that the new password is different from the one provided. The data passed into the function is encrypted along with the modified new password, then logged so a defender can get alerted about the change as well as decrypt the new password for use.
Alter the output of an adversary's profiling commands to make newly-built systems look like the operating system was installed months earlier.
Alter the output of adversary recon commands to not show important assets, such as a file server containing sensitive data.
